In [9]:
from h5glance import H5Glance
import h5py

In [15]:
SIM = h5py.File("/user/vitaldehenau/Github/MGMR/run/TestCase2.hdf5")
H5Glance(SIM)

/user/vitaldehenau/Github/MGMR/run/TestCase2.hdf5/
├atmosphere	(3 children)
├inputs	(0 children) (30 attributes)
└observers	(632 children)

In [41]:
print(list(SIM["/inputs"].attrs))

print(" ")
print(list(SIM["/observers/pos_010_045"].attrs))
print((SIM["/observers/pos_010_045"].attrs["columns"]))
print((SIM["/observers/pos_010_045"].attrs["units"]))

print(" ")
print(list(SIM["/atmosphere/Currents"].attrs))
print(list(SIM["/atmosphere/Currents"].attrs["columns"]))
print(list(SIM["/atmosphere/Currents"].attrs["units"]))
print(list(SIM["/atmosphere/Force"].attrs["columns"]))
print(list(SIM["/atmosphere/Force"].attrs["units"]))
print(list(SIM["/atmosphere/Geometry"].attrs["columns"]))
print(list(SIM["/atmosphere/Geometry"].attrs["units"]))


['AlternativeSmooth', 'D_ESmooth', 'ENERGY_SH2', 'F_lim', 'Fit_StI', 'GROUNDLEVEL', 'IntegrateCurrent', 'Intensity_Weight', 'Intensity_weight', 'J0Q', 'J0T', 'MOLIERERADIUS', 'ObsDist_dim', 'OutFileLabel', 'RnrmA', 'RnrmB', 'SAMPLINGTIME', 'StParRange', 'X_02', 'X_MAX2', 'X_max', 'azi_B', 'energy_sh', 'lamx', 'lamx2', 'nu_max', 'nu_min', 'u0', 'x_0', 'zen_B']
 
['columns', 'units']
[b't' b'Re_Ex' b'Im_Ex' b'Re_Ey' b'Im_Ey']
[b'ns' b'V/m' b'V/m' b'V/m' b'V/m']
 
['columns', 'units']
[b'J_x', b'J_y', b'J_Q', b'dxi']
[b'None', b'None', b'None', b'g/cm^3]']
[b'Fx', b'Fy', b'|F|', b'phi']
[b'km', b'km', b'keV/m', b'degree']
[b'z', b'X', b'refractivity', b'alpha(z)']
[b'km', b'g/cm^2', b'None', b'None']


In [20]:
SIM = h5py.File("/user/vitaldehenau/MA_simulations/CODE/HDF5/SIM000064.hdf5")
H5Glance(SIM)

/user/vitaldehenau/MA_simulations/CODE/HDF5/SIM000064.hdf5/ (2 attributes)
├CoREAS	(1 children) (30 attributes)
├Double Bump	(0 children) (7 attributes)
├atmosphere	(2 children) (1 attributes)
└inputs	(0 children) (12 attributes)

In [21]:
print((SIM["/CoREAS/observers/pos_100_0_8"]))


<HDF5 dataset "pos_100_0_8": shape (2082, 4), type "<f8">
